<p><h1> Manipulating Time Series in ODM2 </h1></p>
<p>This is a sample Jupyter Notebook for ODM2 and ODM2 Admin. This file contains code to make some modifications to some time series. Removing some duplicate values and adding null values where there is a break in the time series.</p> 
<p>1) In the first block of code below, we import the Django ORM models and model helper functions. </p>
<p>2) Use the model helper function searchModelColumnFor to find a sampling feature (location) with a particular name. The parameters are 1- the string to search for, 2- the Django ORM model to search in, 3- the model field to search in. </p>

In [1]:
from ODM2CZOData.models import *
from ODM2CZOData.modelHelpers import *
Sonadora = searchModelColumnFor(" QS-Quebrada Sonadora ", Samplingfeatures, 'samplingfeaturecode')

<p>3) Next find the feature actions related to this sampling feature, the results related to it, then the time series results associated with the results.</p>

In [2]:
SonadoraFeatureActions = Featureactions.objects.filter(samplingfeatureid=Sonadora)
SonadoraResults = Results.objects.filter(featureactionid__in=SonadoraFeatureActions)
SonadoraTimeSeriesResults = Timeseriesresults.objects.filter(resultid__in=SonadoraResults)

<p>4) Now display the time series results and make sure they are the ones your looking for. </p>

In [3]:
for SonadoraTimeSeriesResult in SonadoraTimeSeriesResults:
    print(SonadoraTimeSeriesResult)

Abs Pres - Water Pressure gauge - 1698- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | Hobo U20 Water Level , Observation - ID: 16154 , 15.0, Minutes- Minutes
Absorbance-254 - Light absorbed from  - 1699- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | SUNA , Observation - ID: 16543 , 15.0, Minutes- Minutes
Absorbance-350 - Light Absorbed from  - 1699- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | SUNA , Observation - ID: 16544 , 15.0, Minutes- Minutes
Bromide - Bromide - 1699- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | SUNA , Observation - ID: 16546 , 15.0, Minutes- Minutes
CDOM - Organic matter in wa - 1700- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | Manta-multi probe , Observation - ID: 16537 , 15.0, Minutes- Minutes
CO2- millivolts - millivolts from Vail - 1701- QS-Quebrada Sonadora - 3- Stream gage - Quebrada Sonadora - Observation | Vaisal

<p> 5) Next find the time series result values related to these time series results and order them by result and time. </p> 

In [4]:
tsrvs = Timeseriesresultvalues.objects.filter(resultid__in=SonadoraTimeSeriesResults).filter(valuedatetime__gte='2016-11-06')\
        .order_by('resultid', 'valuedatetime')

C:\Users\leonmi\AppData\Roaming\Python\Python27\site-packages\django\db\models\fields\__init__.py:1430: RuntimeWarning: DateTimeField Timeseriesresultvalues.valuedatetime received a naive datetime (2016-11-06 00:00:00) while time zone support is active.
  RuntimeWarning)


<p>6) find duplicate time series result values and delete them. </p>

In [5]:
lasttsrv = None
for tsrv in tsrvs:
    if lasttsrv == None:
        lasttsrv = tsrv
        continue
    elif tsrv.resultid == lasttsrv.resultid and tsrv.valuedatetime == lasttsrv.valuedatetime:
        print(tsrv)
        #tsrv.delete()
    lasttsrv = tsrv

<p>7) Now, for each time series, we will insert NAN values, during a time period when data was not being recorded. This will allow highcharts in ODM2 Admin to display the graphs properly, with a gap.  </p>

In [9]:
censorcode = CvCensorcode.objects.filter(name="Not censored").get()
qualitycode = CvQualitycode.objects.filter(name="Good").get()

for SonadoraTimeSeriesResult in SonadoraTimeSeriesResults:
    newtsrv = Timeseriesresultvalues(resultid=SonadoraTimeSeriesResult, datavalue=None,)
    Timeseriesresultvalues(
                                                    resultid=SonadoraTimeSeriesResult,
                                                    datavalue='NAN',
                                                    valuedatetime='2016-11-26 12:00:00',
                                                    valuedatetimeutcoffset=4,
                                                    censorcodecv=censorcode,
                                                    qualitycodecv=qualitycode,
                                                    timeaggregationinterval=SonadoraTimeSeriesResult
                                                    .intendedtimespacing,
                                                    timeaggregationintervalunitsid=SonadoraTimeSeriesResult
                                                    .intendedtimespacingunitsid
                                                )#.save()